In [363]:
FORM_MAPS = {
    "le génitif": "genitive",
    "gen.": "genitive",
    "gén.": "genitive",
    "gén": "genitive",
    "dat.": "dative",
    "dat": "dative",
    "plur.": "plural",
    "plur": "plural",
    "futur": "future",
    "fut.": "future",
    "participe": "participle",
    "comp.": "comparative",
    "comp": "comparative",
    "impér.": "imperative",
    "impér": "imperative"
}

In [385]:
import json
with open("sjoestedt_phonetique.json") as inf:
    DATA = json.load(inf)

In [349]:
section = 30

if not section in DATA:
    DATA[section] = []

In [384]:
DATA[section] = []

In [364]:
import re
REGEX_CASE = rf"^([^(]+) \(([^)]+)\) « ([^»]+) », ({'|'.join(FORM_MAPS.keys())}) ([^(]+) \(([^)]+)\)$"
REGEX_CASE_OF = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) « ([^»]+) »$"
REGEX_CASE_OF_BOTH = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_BASE = r"^([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_JUST_FRENCH = r"^([^(]+) « ([^»]+) »$"
REGEX_FROM = r"^([^(]+) \(([^)]+)\) « ([^»]+) »,? de ([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_IN = r"^([^(]+) \(([^)]+)\), dans ([^«]+) « ([^»]+) »$"
REGEX_OR = r"^([^(]+) \(([^)]+)\) ou ([^(]+) \(([^)]+)\) « ([^»]+) »$"

def extract(text, splitter, counter):
    text = text.replace("\u00ad", "").replace("\n", " ")
    for p in text.split(splitter):
            m = re.match(REGEX_BASE, p.strip())
            mjf = re.match(REGEX_JUST_FRENCH, p.strip())
            mf = re.match(REGEX_FROM, p.strip())
            min = re.match(REGEX_IN, p.strip())
            mc = re.match(REGEX_CASE, p.strip())
            mcofb = re.match(REGEX_CASE_OF_BOTH, p.strip())
            mcof = re.match(REGEX_CASE_OF, p.strip())
            mor = re.match(REGEX_OR, p.strip())
            if m:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": m.group(1),
                    "irish": m.group(2),
                    "french": m.group(3),
                })
                counter += 1
            elif mjf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mjf.group(1),
                    "french": mjf.group(2),
                })
                counter += 1
            elif mf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(1),
                    "irish": mf.group(2),
                    "french": mf.group(3).replace("\xad", ""),
                    "from": mf.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(4),
                    "irish": mf.group(5),
                    "french": mf.group(6).replace("\xad", ""),
                })
                counter += 1
            elif mor:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(1),
                    "irish": mor.group(2),
                    "french": mor.group(5),
                    "alt": mor.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(3),
                    "irish": mor.group(4),
                    "french": mor.group(5),
                    "alt": mor.group(2)
                })
                counter += 1
            elif min:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(1),
                    "irish": min.group(2),
                    "in": min.group(3)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(3),
                    "french": min.group(4),
                })
                counter += 1
            elif mc:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(1),
                    "irish": mc.group(2),
                    "french": mc.group(3),
                    FORM_MAPS[mc.group(4)]: mc.group(6)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(5),
                    "irish": mc.group(6),
                    f"{FORM_MAPS[mc.group(4)]}_of": mc.group(2)
                })
                counter += 1
            elif mcofb:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcofb.group(1),
                    "irish": mcofb.group(2),
                    f"{FORM_MAPS[mcofb.group(3)]}_of": mcofb.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "irish": mcofb.group(5),
                    "transcription": mcofb.group(4),
                    "french": mcofb.group(6),
                    FORM_MAPS[mcofb.group(3)]: mcofb.group(2)
                })
                counter += 1
            elif mcof:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(1),
                    "irish": mcof.group(2),
                    f"{FORM_MAPS[mcof.group(3)]}_of": mcof.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(4),
                    "french": mcof.group(5),
                    FORM_MAPS[mcof.group(3)]: mcof.group(2)
                })
                counter += 1
            else:
                print(p)
    print(counter)


bəkaꞏχ (bacach) « boiteux » ; bɑlkʷɩrʹɩ (balcaire) « homme trapu, rustaud » ; fo̤kəl (focal) « mot » ; glɑkʷɩmʹ (glacaim) « je prends » ; fɑ꞉kʷɩ ʃeꞏ (fágfaidh sé) « il laissera » de fɑ꞉gʷɩmʹ (fágaim) « je laisse » tᴜkʷɩ ʃeꞏ (tiocfaidh sé) « il viendra » ; fɑ꞉kə ou fɑ꞉kəhə (fágtha) « laissé » ; tɑkəhə (tagtha) « venu ».

aɪᵊrk (adharc) « corne » ; kʹαrk (cearc) « poule » ; mᴜk (muc) « cochon » ; ɔlk (olc) « mauvais » ; tɑrək (tarrang) « tirer » ; sᴜk, sᴜk (suc, suc), « appel pour les veaux »

In [386]:
# DATA[section] = []
section = 36
if not section in DATA:
    DATA[section] = []
TEXT = "bəkaꞏχ (bacach) « boiteux » ; bɑlkʷɩrʹɩ (balcaire) « homme trapu, rustaud » ; fo̤kəl (focal) « mot » ; glɑkʷɩmʹ (glacaim) « je prends » ; fɑ꞉kʷɩ ʃeꞏ (fágfaidh sé) « il laissera » de fɑ꞉gʷɩmʹ (fágaim) « je laisse » ; tᴜkʷɩ ʃeꞏ (tiocfaidh sé) « il viendra » ; fɑ꞉kə ou fɑ꞉kəhə (fágtha) « laissé »"
extract(TEXT, ";", 12)

20


In [387]:
DATA

{'6': [{'section': 6,
   'id': '6_1',
   'transcription': 'fʹαr',
   'irish': 'fear',
   'french': 'homme',
   'genitive': 'fir'},
  {'section': 6,
   'id': '6_2',
   'transcription': 'fʹirʹ',
   'irish': 'fir',
   'genitive_of': 'fear'},
  {'section': 6,
   'id': '6_3',
   'transcription': 'pɑkə',
   'irish': 'paca',
   'french': 'sac'},
  {'section': 6,
   'id': '6_4',
   'transcription': 'pʹαkə',
   'irish': 'peaca',
   'french': 'péché'},
  {'section': 6,
   'id': '6_5',
   'transcription': 'bo꞉',
   'irish': 'bó',
   'french': 'vache'},
  {'section': 6,
   'id': '6_6',
   'transcription': 'bʲo꞉',
   'irish': 'beó',
   'french': 'vivant'},
  {'section': 6,
   'id': '6_7',
   'transcription': 'tᴀᴜn',
   'irish': 'tonn',
   'french': 'une vague'},
  {'section': 6,
   'id': '6_8',
   'transcription': 'tʲaᴜn',
   'irish': 'teann',
   'french': 'trapu'},
  {'section': 6,
   'id': '6_9',
   'transcription': 'do꞉',
   'irish': 'dóghadh',
   'french': 'brûler'},
  {'section': 6,
   'id': '

In [380]:
DATA[section][1]["transcription"] = DATA[section][1]["transcription"].split(" ou ")
DATA[section][2]["transcription"] = DATA[section][2]["transcription"].split(" ou ")

In [382]:
import json
with open("sjoestedt_phonetique.json", "w") as outf:
    json.dump(DATA, outf)